In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Bladder_Cancer/GSE244266'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Association of Molecular Subtypes with Pathologic Response, PFS and OS in a Phase II Study of Coexpression Extrapolation (COXEN) with Neoadjuvant Chemotherapy (NAC) for Localized, Muscle-Invasive Bladder Cancer (SWOG S1314; NCT02177695)"
!Series_summary	"Investigation of RNA-based molecular subtypes as additional predictive biomarkers for neoadjuvant chemotherapy response, progression-free survival and survival in patients treated in S1314."
!Series_overall_design	"randomized 2-arm phase II trial of neoadjuvant chemotherapy in muscle-invasive bladder cancer, translational analysis of tissue"
Sample Characteristics Dictionary:
{0: ['treatment arm: DDMVAC+CYST', 'treatment arm: GC+CYST'], 1: ['disease: muscle-invasive bladder cancer'], 2: ['clinical_stage_strat_factor: Clinical Stage -T2', 'clinical_stage_strat_factor: Clinical Stage -T3 or T4a']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Check if the dataset contains gene expression data
# Given that the data is RNA-based, set is_gene_available to True
is_gene_available = True

# Variable Availability and Data Type Conversion
# Based on the provided sample characteristics dictionary, the relevant variables are inferred as:
# - Bladder_Cancer: Not explicitly listed but assumed as all samples have 'muscle-invasive bladder cancer'
# - Age and Gender: Not available in the given dictionary

trait_row = None  # No specific entry for 'Bladder_Cancer' as all cases are muscle-invasive bladder cancer  
age_row = None  # Age information is not available in the sample characteristics dictionary
gender_row = None  # Gender information is not available in the sample characteristics dictionary

# Conversion functions assuming standard format, though not applicable here since rows are None
def convert_trait(value):
    try:
        val = value.split(':')[-1].strip().lower()
        return 1 if 'bladder cancer' in val else 0
    except:
        return None

def convert_age(value):
    try:
        val = value.split(':')[-1].strip()
        return float(val)
    except:
        return None

def convert_gender(value):
    try:
        val = value.split(':')[-1].strip().lower()
        return 1 if 'male' in val else 0 if 'female' in val else None
    except:
        return None

# Save Metadata
save_cohort_info('GSE244266', './preprocessed/Bladder_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
# Since trait_row is None, no clinical feature extraction is performed.


A new JSON file was created at: ./preprocessed/Bladder_Cancer/cohort_info.json
